In [7]:
#!pip install lyricsgenius python-dotenv pandas

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import lyricsgenius as lg
import re
import time

In [2]:
load_dotenv()
GENIUS_TOKEN = os.getenv("GENIUS_ACCESS_TOKEN")

if not GENIUS_TOKEN:
    raise ValueError("Токен GENIUS_ACCESS_TOKEN не найден в .env файле или переменных окружения.")

In [3]:
genius = lg.Genius(
    GENIUS_TOKEN,
    verbose=False,
    remove_section_headers=True,
    skip_non_songs=False,                   # оставляем возможность для фитов (нам это нужно)
    excluded_terms=["(Live)", "(Remix)"],   # фильтруем шум
    timeout=30
)

In [4]:
def clean_lyrics(text):
    if not isinstance(text, str):
        return ""
    cleaned_text = text

    # 1. удаляем артефакты Genius
    cleaned_text = re.sub(r'Embed$', '', cleaned_text, flags=re.MULTILINE).strip()
    cleaned_text = re.sub(r'^\d+Embed$', '', cleaned_text, flags=re.MULTILINE).strip()

    # 2. нормализация переносов строк
    cleaned_text = re.sub(r'\n{3,}', '\n\n', cleaned_text).strip()

    # 3. удаление всех нестандартных символов и невидимых пробелов (ключ к решению 4/MSP)
    cleaned_text = re.sub(r'[^\w\s\.,:;?!а-яА-ЯёЁ]+', ' ', cleaned_text)

    # 4. удаление специфического артефакта типа '4/MSP'
    cleaned_text = re.sub(r'\s*\d+/\w+\s*', ' ', cleaned_text, flags=re.IGNORECASE)

    # 5. нормализация пробелов (множественные -> одиночный)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

In [12]:
ARTIST_LIST = ["Boulevard Depo", "SALUKI", "Хаски", "Kai Angel", "PHARAOH", "FRIENDLY THUG 52 NGG", "ROCKET", "Mnogoznaal", "Toxi$", 'MAYOT', "LILDRUGHILL"]
MAX_SONGS = None
all_lyrics = []
for ARTIST_NAME in ARTIST_LIST:
    print(f"\n Начинаем сбор песен артиста: {ARTIST_NAME}. Это может занять несколько минут.")

    try:
        # шаг 1: Поиск артиста
        artist_obj = genius.search_artist(ARTIST_NAME, max_songs=MAX_SONGS, sort='popularity')

        if not artist_obj:
            print(f" Артист '{ARTIST_NAME}' не найден.")
            continue

        print(f" Найдено {len(artist_obj.songs)} песен.")

        # шаг 2: Итерация по песням и очистка (ВНУТРИ цикла артистов)
        for song in artist_obj.songs:

            # 1. извлечение имени основного артиста (для фильтрации)
            primary_artist_name = song.primary_artist.get('name', ARTIST_NAME) \
                                  if isinstance(song.primary_artist, dict) else ARTIST_NAME

            # 2. извлечение чистого имени альбома
            album_name = None
            if isinstance(song.album, dict) and 'name' in song.album:
                album_name = song.album['name']

            # 3. очистка текста (первичная - удаление артефактов Genius)
            lyrics_text = song.lyrics
            lyrics_text = clean_lyrics(lyrics_text)

            all_lyrics.append({
                'artist': ARTIST_NAME,
                'title': song.title,
                'album': album_name,
                'lyrics': lyrics_text,
                'primary_artist': primary_artist_name
            })

        # пауза, чтобы не перегружать API Genius
        time.sleep(2)

    except Exception as e:

        print(f"Произошла ошибка при сборе данных для {ARTIST_NAME}: {e}")


 Начинаем сбор песен артиста: Boulevard Depo. Это может занять несколько минут.
 Найдено 209 песен.

 Начинаем сбор песен артиста: SALUKI. Это может занять несколько минут.
 Найдено 192 песен.

 Начинаем сбор песен артиста: Husky. Это может занять несколько минут.
 Найдено 50 песен.

 Начинаем сбор песен артиста: Kai Angel. Это может занять несколько минут.
 Найдено 113 песен.

 Начинаем сбор песен артиста: PHARAOH. Это может занять несколько минут.
 Найдено 260 песен.

 Начинаем сбор песен артиста: FRIENDLY THUG 52 NGG. Это может занять несколько минут.
 Найдено 100 песен.

 Начинаем сбор песен артиста: ROCKET. Это может занять несколько минут.
 Найдено 130 песен.

 Начинаем сбор песен артиста: Mnogoznaal. Это может занять несколько минут.
 Найдено 96 песен.

 Начинаем сбор песен артиста: Toxi$. Это может занять несколько минут.
 Найдено 104 песен.

 Начинаем сбор песен артиста: MAYOT. Это может занять несколько минут.
 Найдено 488 песен.

 Начинаем сбор песен артиста: LILDRUGHILL. Э

In [ ]:
DATA_FOLDER = "data"
if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)

df = pd.DataFrame(all_lyrics)
df = df[df['lyrics'].str.len() > 10].copy()
# df.info()

In [ ]:
OUTPUT_FILE = f'{DATA_FOLDER}/lyrics_api.csv'
df.to_csv(OUTPUT_FILE, index=False)
print(f"Данные сохранены в {OUTPUT_FILE}. Итоговое количество чистых песен: {len(df)}")

In [15]:
df_copy = df.copy()
df_copy['char_count'] = df['lyrics'].apply(len)
df_copy['word_count'] = df['lyrics'].apply(lambda x: len(x.split()))

print(df_copy['char_count'].describe())
print('----------------')
print(df_copy['word_count'].describe())

count    1852.000000
mean     1307.337473
std       629.796347
min        23.000000
25%       888.750000
50%      1297.500000
75%      1701.000000
max      4548.000000
Name: char_count, dtype: float64
----------------
count    1852.000000
mean      252.365011
std       122.319796
min         5.000000
25%       173.500000
50%       251.000000
75%       328.000000
max       896.000000
Name: word_count, dtype: float64
